In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import torch
import plotly.graph_objs as go
from torch_geometric.data import Data
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np


In [ ]:

# Define joints and edges
joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
    'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
    'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
    'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
    'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

edges = [
    ('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
    ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
    ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
    ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
    ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
    ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
    ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
    ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
    ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
    ('HEAD', 'NOSE'),('HEAD','EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')
]

joint_to_idx = {joint: idx for idx, joint in enumerate(joints)}

edge_index = torch.tensor(
    [[joint_to_idx[src], joint_to_idx[dst]] for src, dst in edges] +
    [[joint_to_idx[dst], joint_to_idx[src]] for src, dst in edges],  # Bidirectional edges
    dtype=torch.long
).t()

# Custom Dataset Class
class SkeletonDataset(Dataset):
    def __init__(self, csv_file):
        self.data = csv_file
        self.num_nodes = len(joints)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Extract node features and include timestamp
        node_features = []
        for joint in joints:
            x = row[f'{joint}_X']
            y = row[f'{joint}_Y']
            z = row[f'{joint}_Z']
            timestamp = row['timeStamps']  # Include timestamp as a feature
            node_features.append([x, y, z, timestamp])

        x = torch.tensor(node_features, dtype=torch.float)  # Shape: [num_nodes, 4]

        # Label
        y = torch.tensor(row['frailty_class'], dtype=torch.float)

        # Create data object
        data = Data(x=x, edge_index=edge_index, y=y)

        return data


In [ ]:


# Hybrid STGCN Model Definition
class HybridSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super(HybridSTGCN, self).__init__()

        # Spatial Graph Convolution Layers
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)

        # Temporal Convolution Layers
        self.temporal_conv1 = nn.Conv1d(in_channels=hidden_channels,
                                        out_channels=hidden_channels,
                                        kernel_size=3, padding=1)

        # LSTM for temporal modeling
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)

        # Fully Connected Layer
        self.fc = nn.Linear(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch  # x: [num_nodes*batch_size, in_channels]

        # Spatial GCN Layers
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))  # x: [num_nodes*batch_size, hidden_channels]

        # Reshape for Temporal Conv
        batch_size = batch.max().item() + 1
        num_nodes = x.size(0) // batch_size
        x = x.view(batch_size, num_nodes, -1)  # [batch_size, num_nodes, hidden_channels]
        x = x.permute(0, 2, 1)  # [batch_size, hidden_channels, num_nodes]

        # Temporal Conv Layer
        x = F.relu(self.temporal_conv1(x))  # [batch_size, hidden_channels, num_nodes]

        # LSTM for capturing temporal patterns
        x = x.permute(0, 2, 1)  # [batch_size, num_nodes, hidden_channels]
        x, _ = self.lstm(x)  # [batch_size, num_nodes, hidden_channels]

        # Pooling over nodes
        x = x.mean(dim=1)  # [batch_size, hidden_channels]

        # Fully Connected Layer
        x = self.fc(x)  # [batch_size, num_classes]
        x = torch.sigmoid(x)  # For binary classification

        return x.squeeze()  # [batch_size]



In [ ]:

data_path = '/content/drive/MyDrive/Iqram Sir/AI_Frailty-MATLAB_Sample/Final_data_Balanced.csv'

dataframe = pd.read_csv(data_path)





# Load dataset
dataset = SkeletonDataset(csv_file=dataframe[dataframe['walking_speed'] == "Fast"])

# Split into training and testing datasets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
dataframe

,patientID,trial_num,walking_speed,timeStamps,PELVIS_X,SPINE_NAVAL_X,SPINE_CHEST_X,NECK_X,CLAVICLE_LEFT_X,SHOULDER_LEFT_X,...,KNEE_RIGHT_Z,ANKLE_RIGHT_Z,FOOT_RIGHT_Z,HEAD_Z,NOSE_Z,EYE_LEFT_Z,EAR_LEFT_Z,EYE_RIGHT_Z,EAR_RIGHT_Z,frailty_class
0,001-LO,Trial1,Regular,0.014806,513.079538,516.003537,527.994788,545.712851,516.264630,405.175317,...,929.656427,825.884006,894.117256,925.016797,954.213183,938.401915,886.661529,968.354628,981.439603,1
1,001-LO,Trial1,Regular,0.034787,518.341956,523.496677,539.288301,549.834306,520.128180,405.456134,...,924.690502,784.109292,847.342565,961.514530,951.963861,963.009239,945.312977,991.238931,1044.996519,1
2,001-LO,Trial1,Regular,0.054767,529.923963,541.248914,552.630499,552.997228,524.350174,408.479201,...,936.771136,777.246423,837.827719,987.262010,983.921111,995.232664,973.751899,1023.415136,1073.302525,1
3,001-LO,Trial1,Regular,0.074748,534.861455,548.038017,556.688923,551.944125,523.861126,407.510892,...,967.044120,802.468369,862.476378,1015.462905,1023.426646,1033.121369,1005.673693,1059.598333,1098.709654,1
4,001-LO,Trial1,Regular,0.094728,534.057534,545.354058,552.725394,547.248025,519.301345,403.093700,...,1013.036813,855.414270,916.645154,1045.783754,1069.454529,1075.905348,1040.604981,1099.243905,1121.611912,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43468,071-EK,Trial4,Fast,5.223442,139.765899,131.899616,124.789803,129.198960,159.941471,284.639251,...,950.235146,1125.552457,1069.957645,661.130624,682.153747,690.918415,650.418190,722.157079,759.024619,0
43469,071-EK,Trial4,Fast,5.243423,141.382626,132.692612,125.146661,127.533644,158.607992,283.347906,...,933.171147,1088.115939,1019.434259,654.994076,679.151059,686.343676,642.756755,718.257095,753.004001,0
43470,071-EK,Trial4,Fast,5.263403,142.999353,133.485609,125.503519,125.868327,157.274514,282.056561,...,916.107147,1050.679421,968.910873,648.857528,676.148370,681.768936,635.095321,714.357112,746.983384,0
43471,071-EK,Trial4,Fast,5.283384,144.616080,134.278605,125.860377,124.203011,155.941036,280.765215,...,899.043147,1013.242903,918.387487,642.720979,673.145682,677.194197,627.433886,710.457128,740.962766,0


In [ ]:
# Initialize Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridSTGCN(in_channels=4, hidden_channels=128, num_classes=1).to(device)

In [ ]:
# Define Loss Function and Optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
# Training Function
def train(epoch):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)  # [batch_size]
        y = data.y.to(device).float()  # [batch_size]

        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.num_graphs

    avg_loss = total_loss / len(train_loader.dataset)
    print(f'Epoch {epoch}, Loss: {avg_loss:.4f}')

In [ ]:
# Evaluation Function
def evaluate(loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            preds = (out >= 0.5).float()
            y = data.y.to(device)
            correct += (preds == y).sum().item()
            total += data.num_graphs
    accuracy = correct / total
    return accuracy


In [ ]:


# Training Loop
num_epochs = 30

for epoch in range(1, num_epochs + 1):
    train(epoch)
    train_acc = evaluate(train_loader)
    test_acc = evaluate(test_loader)
    print(f'Epoch {epoch}, Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}')

Epoch 1, Loss: 0.6488
Epoch 1, Train Accuracy: 0.6229, Test Accuracy: 0.6157
Epoch 2, Loss: 0.5875
Epoch 2, Train Accuracy: 0.6722, Test Accuracy: 0.6561
Epoch 3, Loss: 0.5502
Epoch 3, Train Accuracy: 0.7005, Test Accuracy: 0.6910
Epoch 4, Loss: 0.5290
Epoch 4, Train Accuracy: 0.7422, Test Accuracy: 0.7237
Epoch 5, Loss: 0.5052
Epoch 5, Train Accuracy: 0.7474, Test Accuracy: 0.7381
Epoch 6, Loss: 0.4832
Epoch 6, Train Accuracy: 0.7779, Test Accuracy: 0.7600
Epoch 7, Loss: 0.4628
Epoch 7, Train Accuracy: 0.7785, Test Accuracy: 0.7620
Epoch 8, Loss: 0.4455
Epoch 8, Train Accuracy: 0.7969, Test Accuracy: 0.7782


KeyboardInterrupt: 

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

def visualize_dataset(dataset):
    # Initialize the plotly figure
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

    # Loop through each graph in the dataset to create frames
    frames = []
    for i, data in enumerate(dataset):
        node_positions = data.x[:, :3].numpy()  # Extract (x, y, z) coordinates, ignoring the timestamp

        # Extract x, y, z coordinates from node_positions
        x_coords = node_positions[:, 0]
        y_coords = node_positions[:, 1]
        z_coords = node_positions[:, 2]

        # Create edges for Plotly
        edge_x = []
        edge_y = []
        edge_z = []
        for edge in edge_index.t().tolist():
            x0, y0, z0 = node_positions[edge[0]]
            x1, y1, z1 = node_positions[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
            edge_z.extend([z0, z1, None])

        # Plot the edges
        edge_trace = go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            mode='lines',
            line=dict(color='black', width=2),
            hoverinfo='none'
        )

        # Plot the nodes with joint names
        node_trace = go.Scatter3d(
            x=x_coords, y=y_coords, z=z_coords,
            mode='markers+text',
            marker=dict(size=6, color='blue'),
            text=joints,  # Use the joint names as text labels
            hoverinfo='text'
        )

        # Add the frame
        frames.append(go.Frame(data=[edge_trace, node_trace], name=str(i)))

    # Create the initial plot with the first graph
    fig.add_trace(frames[0].data[0])
    fig.add_trace(frames[0].data[1])

    # Update the layout with sliders and animation settings
    fig.update_layout(
        title='3D Graph Animation of Dataset',
        scene=dict(
            xaxis=dict(showbackground=True, backgroundcolor="rgb(230, 230, 230)", gridcolor="rgb(200, 200, 200)", showgrid=True, zerolinecolor="rgb(200, 200, 200)"),
            yaxis=dict(showbackground=True, backgroundcolor="rgb(230, 230, 230)", gridcolor="rgb(200, 200, 200)", showgrid=True, zerolinecolor="rgb(200, 200, 200)"),
            zaxis=dict(showbackground=True, backgroundcolor="rgb(230, 230, 230)", gridcolor="rgb(200, 200, 200)", showgrid=True, zerolinecolor="rgb(200, 200, 200)")
        ),
        updatemenus=[dict(type='buttons', showactive=False,
                          buttons=[dict(label='Play',
                                        method='animate',
                                        args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True, mode='immediate')]),
                                   dict(label='Pause',
                                        method='animate',
                                        args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])])],
        sliders=[{
            'steps': [{'args': [[f.name], dict(mode='immediate', frame=dict(duration=500, redraw=True))],
                       'label': str(k), 'method': 'animate'} for k, f in enumerate(frames)],
            'currentvalue': {'prefix': 'Frame: ', 'font': {'size': 20}},
            'pad': {'b': 10, 't': 50},
            'len': 0.9, 'x': 0.1, 'y': 0,
        }],
        width=1200, height=800, margin=dict(l=0, r=0, b=0, t=40)
    )

    # Add all frames to the figure
    fig.frames = frames

    fig.show()


visualize_dataset(dataset)



In [ ]:
import torch
from torch_geometric.nn import GNNExplainer
import matplotlib.pyplot as plt

# Assuming your model is defined as HybridSTGCN and trained
model = HybridSTGCN(in_channels=4, hidden_channels=64, num_classes=1)
model.load_state_dict(torch.load('model_checkpoint.pth'))  # Load your trained model
model.eval()  # Set the model to evaluation mode

# Initialize the GNNExplainer
explainer = GNNExplainer(model, epochs=200, return_type='prob')

# Explain a specific graph-level prediction
data = dataset[0]  # Use the first graph in the dataset (or any other graph)
node_feat_mask, edge_mask = explainer.explain_graph(data.x, data.edge_index)

# Visualize the explanation
ax, G = explainer.visualize_subgraph(node_idx=None, edge_index=data.edge_index, edge_mask=edge_mask, y=data.y)
plt.show()


ImportError: cannot import name 'GNNExplainer' from 'torch_geometric.nn' (/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/__init__.py)

In [ ]:
!pip uninstall torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv

Found existing installation: torch_geometric 2.5.3
Uninstalling torch_geometric-2.5.3:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torch_geometric-2.5.3.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch_geometric/*
Proceed (Y/n)? Y
  Successfully uninstalled torch_geometric-2.5.3


In [ ]:
!pip install torch
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from torch_geometric.nn import GNNExplainer


ImportError: cannot import name 'GNNExplainer' from 'torch_geometric.nn' (/usr/local/lib/python3.10/dist-packages/torch_geometric/nn/__init__.py)

In [ ]:
pip install shap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 9.8 MB/s eta 0:00:00


In [ ]:
import torch
from torch_geometric.nn import GCNConv
import shap

# Assume your model is already trained and defined as `model`
model = HybridSTGCN(in_channels=4, hidden_channels=64, num_classes=1)
model.eval()  # Set the model to evaluation mode


HybridSTGCN(
  (gcn1): GCNConv(4, 64)
  (gcn2): GCNConv(64, 64)
  (temporal_conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (lstm): LSTM(64, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
def gnn_predict(node_features_list):
    model.eval()
    preds = []
    for node_features in node_features_list:
        data = Data(x=torch.tensor(node_features, dtype=torch.float), edge_index=edge_index)
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
            preds.append(out.item())  # Assuming binary classification
    return np.array(preds)


In [ ]:
# Convert the dataset into a list of node feature arrays (ignoring edge features for SHAP)
background = [data.x.numpy() for data in dataset[:100]]  # Use a subset of data for SHAP background


TypeError: Unknown type passed as data object: <class 'torch_geometric.data.data.Data'>